In [1]:
!pip install openai-whisper
!pip install nltk
!pip install transformers
!pip install sentence-transformers


In [2]:
!pip install language-tool-python

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anage\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\anage\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import whisper
import nltk
import language_tool_python
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
import numpy as np

c:\Users\anage\anaconda3\envs\byteXL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load models
whisper_model = whisper.load_model("base")
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
tool = language_tool_python.LanguageTool('en-US')

In [6]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\anage\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
from pathlib import Path
import os

def transcribe_audio(file_path):
    file_path = Path(file_path).resolve(strict=True)  # resolve to absolute path
    print("Resolved path -->", file_path)
    print("Exists:", file_path.exists())

    result = whisper_model.transcribe(str(file_path))  # convert Path to string
    return result["text"]


In [8]:
def compute_fluency(text):
    words = word_tokenize(text)
    word_count = len(words)
    sentence_count = max(1, len(nltk.sent_tokenize(text)))
    avg_words_per_sentence = word_count / sentence_count
    fluency_score = min(10.0, avg_words_per_sentence / 2)  # heuristic
    return round(fluency_score, 2)

In [9]:
def compute_vocabulary(text):
    words = word_tokenize(text)
    unique_words = set(words)
    diversity = len(unique_words) / len(words)
    vocab_score = min(10.0, diversity * 10)  # 0.0 to 10.0
    return round(vocab_score, 2)

In [10]:
def compute_grammar(text):
    matches = tool.check(text)
    error_rate = len(matches) / max(1, len(text.split()))
    grammar_score = max(0.0, 10.0 - (error_rate * 50))  # Heuristic penalty
    return round(grammar_score, 2)

In [11]:
def compute_relevance(transcription, topic):
    emb1 = semantic_model.encode(transcription, convert_to_tensor=True)
    emb2 = semantic_model.encode(topic, convert_to_tensor=True)
    similarity = util.cos_sim(emb1, emb2).item()
    return round(similarity * 10, 2)

In [12]:
def analyze_audio(file_path, topic):
    transcription = transcribe_audio(file_path)
    print("\nTranscription:\n", transcription)

    return {
        "fluency": compute_fluency(transcription),
        "vocabulary": compute_vocabulary(transcription),
        "grammar": compute_grammar(transcription),
        "relevance": compute_relevance(transcription, topic)
    }

In [13]:
pwd

'c:\\Users\\anage\\Documents\\vscode_projects\\Complete_Audio_analysis\\Complete_Audio_analysis\\research_notebooks'

In [14]:
import os
#audio_path = "c:/Users/anage/Documents/vscode_projects/Complete_Audio_analysis/Complete_Audio_analysis/input_audio_files/genevieve.wav"
audio_path = "C:/Users/anage/Downloads/speech1.mp3"
print(os.path.exists(audio_path)) 

True


In [15]:
# Example usage
#if __name__ == "__main__":

topic = "The importance of renewable energy"
scores = analyze_audio(audio_path, topic)
print("\nAnalysis Result:")

print(scores)

Resolved path --> C:\Users\anage\Downloads\speech1.mp3
Exists: True


c:\Users\anage\anaconda3\envs\byteXL\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Transcription:
  Your hands lie open in the long fresh grass. The finger points look through like rosy blooms. Your eyes smile peace. The pasture gleams and glooms beneath billowing skies that scatter and amass. All round our nest far as the eye can pass. Our golden king cup feels with silver edge where the cow partially skirts the Hawthorn hedge to his visible silence still is the hourglass. Your hands lie open in the long fresh grass. The finger points look through like rosy blooms. Your eyes smile peace. The pasture gleams and glooms beneath billowing skies that scatter and amass. All round our nest far as the eye can pass. Our golden king cup feels with silver edge where the cow partially skirts the Hawthorn hedge to his visible silence still is the hourglass.

Analysis Result:
{'fluency': 6.08, 'vocabulary': 4.18, 'grammar': 10.0, 'relevance': 0.6}
